In [3]:
import torch
import pandas as pd

In [10]:
df_true = pd.read_pickle('df_true.pkl')
df_false = pd.read_pickle('df_false.pkl')

In [13]:
df_true.loc['Brown bear', 32]

'The second upper molar is smaller than the others, and is usually absent in adults'